In [1]:
import requests
import pandas as pd
import pytz
from sqlalchemy import create_engine
#from sqlalchemy.orm import sessionmaker
from datetime import datetime, timedelta
from utils import read_api_credentials, connect_to_db, load_to_sql


In [2]:
# AUTENTICACIÓN
api_token = read_api_credentials("config.ini", "api_bcra")

headers = {
    'Authorization': f'Bearer {api_token['api_token']}'
}

In [8]:
# OBTENCIÓN Y PREPARACIÓN DE DATAFRAME
def consolidate(endpoint, description):
    """
    Esta función obtiene datos de un endpoint de la API y los convierte en un DataFrame de pandas.

    Args:
        endpoint (str): El endpoint de la API desde donde se extraen los datos.
        concept (str): El concepto o categoría de los datos (ej. 'plazo fijo').

    Returns: 
        DataFrame: Un DataFrame con los datos obtenidos del endpoint de la API, o un dataframe vacío en caso de error.
    """
    url = f'https://api.estadisticasbcra.com{endpoint}'
    response = requests.get(url, headers=headers)
    start = datetime.now(pytz.timezone('America/Buenos_Aires')) - timedelta(days=30)
    end = datetime.now(pytz.timezone('America/Buenos_Aires')) - timedelta(days=1)
    
    if response.status_code == 200:
        print(f'Status code: {response.status_code}')         
        data = response.json()        
        df = pd.DataFrame(data)        
        df.rename(columns={'d': 'Date', 'v': 'Value'}, inplace= True)        
        df['Date'] = pd.to_datetime(df['Date'])
        df['Date'] = df['Date'].dt.tz_localize('America/Buenos_Aires')        
        df['Concept'] = description        
        filtered_df = df[(df['Date'] >= start) & (df['Date'] <= end)]                
        return filtered_df
                
    else: 
        print(f'Failed to fetch data from {endpoint}. Status code:', response.status_code)
        # Retorna un df vacío
        return pd.DataFrame()
        
## Endpoints y Concepts
endpoints = [
    ("/plazo_fijo", "Plazos fijos (m)"),
    ("/depositos", "Depositos (m)"),
    ("/cajas_ahorro", "Cajas Ahorro (m)"),
    ("/cuentas_corrientes", "Cuentas corrientes (m)"),
    ("/usd", "Dolar blue"),
    ("/usd_of", "Dolar oficial")
]
## Lista vacía de endpoints para alojar durante el for
dataframes = []

## Loop "for" que itera sobre la lista de tuplas, llamando a la función "consolidate" para obtener los df y agregarlos a la lista "dataframes" (siempre que la respuesta no sea None o un df vacío) 
for endpoint, description in endpoints:
    df = consolidate(endpoint, description)
    if df is not None and not df.empty:
        dataframes.append(df)
        

## Unificación de dataframes, generando un index nuevo y ordenando las columnas. Si no se obtuvo información, se arroja un mensaje que lo comenta. 
if dataframes:
    df_final = pd.concat(dataframes, ignore_index=True)
    df_final = df_final[['Date', 'Concept', 'Value']]
    print(df_final)
else: 
    print("No se lograron recolectar datos de los endpoints.")
    

Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
                         Date           Concept        Value
0   2024-01-29 00:00:00-03:00  Plazos fijos (m)  22696084.00
1   2024-01-30 00:00:00-03:00  Plazos fijos (m)  22784965.00
2   2024-01-31 00:00:00-03:00  Plazos fijos (m)  23312711.00
3   2024-02-01 00:00:00-03:00  Plazos fijos (m)  23420278.00
4   2024-02-02 00:00:00-03:00  Plazos fijos (m)  23845445.00
..                        ...               ...          ...
101 2024-02-20 00:00:00-03:00     Dolar oficial       837.25
102 2024-02-21 00:00:00-03:00     Dolar oficial       837.85
103 2024-02-22 00:00:00-03:00     Dolar oficial       838.35
104 2024-02-23 00:00:00-03:00     Dolar oficial       838.95
105 2024-02-26 00:00:00-03:00     Dolar oficial       840.65

[106 rows x 3 columns]


In [5]:
!pip install psycopg2-binary

In [6]:
# CONNECTION TO REDSHIFT
engine = connect_to_db("config.ini", "redshift")
with engine.connect() as conn, conn.begin():
    print(type(engine.connect))

2024-02-27 03:20:09,599 - INFO - Conectándose a la base de datos...
2024-02-27 03:20:09,762 - INFO - Conexión a la base de datos establecida exitosamente


<class 'method'>


In [9]:
# CONNECTION TO REDSHIFT
engine = connect_to_db("config.ini", "redshift")

with engine.connect() as conn, conn.begin():

    conn.execute("TRUNCATE TABLE tomasmartindl_coderhouse.stg_bcra")

    load_to_sql(df_final, "stg_bcra", conn, "append")
    
    conn.execute("""
        MERGE INTO tomasmartindl_coderhouse.bcra
        USING tomasmartindl_coderhouse.stg_bcra AS stg
        ON tomasmartindl_coderhouse.bcra.date = stg.date
        AND tomasmartindl_coderhouse.bcra.concept = stg.concept
        WHEN MATCHED THEN
        update tomasmartindl_coderhouse.bcra.value = stg.value
        WHEN NOT MATCHED THEN
        INSERT (tomasmartindl_coderhouse.bcra.date, tomasmartindl_coderhouse.bcra.concept, tomasmartindl_coderhouse.bcra.value)
        VALUES (stg.date, stg.concept, stg.value)             
    """)
    
    #Debug print: si el código SQL se ejecuta sin excepciones, muestra el mensaj de éxito. 
    print("La inserción de datos a PRD validando duplicados, fue lograda con éxito.")

2024-02-27 03:22:23,424 - INFO - Conectándose a la base de datos...
2024-02-27 03:22:23,425 - INFO - Conexión a la base de datos establecida exitosamente
2024-02-27 03:22:26,051 - INFO - Cargando datos en la base de datos...
g:\My Drive\VScode\Coderhouse-DE_Pre-Entrega2-v2\Coder-DE_Pre-Entrega2_v2\utils.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(
2024-02-27 03:22:26,079 - ERROR - Error al cargar los datos en la base de datos: 'Connection' object has no attribute 'cursor'


ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "tomasmartindl_coderhouse"
LINE 7:         update tomasmartindl_coderhouse.bcra.value = stg.val...
                       ^

[SQL: 
        MERGE INTO tomasmartindl_coderhouse.bcra
        USING tomasmartindl_coderhouse.stg_bcra AS stg
        ON tomasmartindl_coderhouse.bcra.date = stg.date
        AND tomasmartindl_coderhouse.bcra.concept = stg.concept
        WHEN MATCHED THEN
        update tomasmartindl_coderhouse.bcra.value = stg.value
        WHEN NOT MATCHED THEN
        INSERT (tomasmartindl_coderhouse.bcra.date, tomasmartindl_coderhouse.bcra.concept, tomasmartindl_coderhouse.bcra.value)
        VALUES (stg.date, stg.concept, stg.value)             
    ]
(Background on this error at: https://sqlalche.me/e/14/f405)